In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import os

In [4]:
#Pesquisando entre algoritmos de threshold/binarização com uma certa eficiencia esperada para imagens com tal nivel de ruido, tendo em vista que é perceptivel que algum método basico conhecido não teria tanto sucesso para o desafio encontrei um algoritmo conhecido por: Binarization Algorithm by Su et al.(http://doi.acm.org/10.1145/1815330.1815351) e com condigo presente em: https://gist.github.com/pebbie/2c17620e60c662950b02c4949b3010f2#file-su-py.
# O método é capaz de filtrar o background estimando o contraste a partir do maximos e mininos locais. O algoritmo foi aplicado a imagens de documentos históricos, o que o indica como ótimo candidato de solução.
# O algoritmo foi adaptado para utilização em meu código.


nfns = [
        lambda x: np.roll(x, -1, axis=0),
        lambda x: np.roll(np.roll(x, 1, axis=1), -1, axis=0),
        lambda x: np.roll(x, 1, axis=1),
        lambda x: np.roll(np.roll(x, 1, axis=1), 1, axis=0),
        lambda x: np.roll(x, 1, axis=0),
        lambda x: np.roll(np.roll(x, -1, axis=1), 1, axis=0),
        lambda x: np.roll(x, -1, axis=1),
        lambda x: np.roll(np.roll(x, -1, axis=1), -1, axis=0)
        ]

def localminmax(img, fns):
    mi = img.astype(np.float64)
    ma = img.astype(np.float64)
    for i in range(len(fns)):
        rolled = fns[i](img)
        mi = np.minimum(mi, rolled)
        ma = np.maximum(ma, rolled)
    result = (ma-mi)/(mi+ma+1e-16)
    return result

def numnb(bi, fns):
    nb = bi.astype(np.float64)
    i = np.zeros(bi.shape, nb.dtype)
    i[bi==bi.max()] = 1
    i[bi==bi.min()] = 0
    for fn in fns:
        nb += fn(i)
    return nb

def rescale(r,maxvalue=255):
    mi = r.min()
    return maxvalue*(r-mi)/(r.max()-mi)

def binarize_Su_et_al(img):
    gfn = nfns
    N_MIN = 4

    
    g = img
    I = g.astype(np.float64)


    cimg = localminmax(I, gfn)
    _, ocimg = cv2.threshold(rescale(cimg).astype(g.dtype), 0, 1, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    E = ocimg.astype(np.float64)


    N_e = numnb(ocimg, gfn)
    nbmask = N_e>0

    E_mean = np.zeros(I.shape, dtype=np.float64)
    for fn in gfn:
        E_mean += fn(I)*fn(E)

    E_mean[nbmask] /= N_e[nbmask]

    E_var = np.zeros(I.shape, dtype=np.float64)
    for fn in gfn:
        tmp = (fn(I)-E_mean)*fn(E)
        E_var += tmp*tmp

    E_var[nbmask] /= N_e[nbmask]
    E_std = np.sqrt(E_var)*.5

    R = np.ones(I.shape)*255
    R[(I<=E_mean+E_std)&(N_e>=N_MIN)] = 0

    return R.astype(np.uint8)




In [83]:
# A solução se deu em três etapas.
# A primeira, responsavel por remover o fundo da imagem (Algorithm by Su et al) e outras duas responsáveis por realizar transformações para alinhamento.
# A primeira etapa de alinhamento foi a etapa de alinhamento horizontal. A ideia de alinhamento partiu da detecção de linha pela transformada probabilistica de Hough. Essa transformada recebe a imagem ja tranformada por outros três filtros: um filtro gaussiano, um filtro sharpen para evidenciar bordas e em seguida um filtro de canny é aplicado.
# Com a detecção das linhas é possivel utilizar uma delas e verificar o alinhamento horizontal entre os dois vertices da reta. Para alinhamento então é aplicado warpAffine o qual realiza rotações de 1 grau na imagem no horario ou anti horaio até que os dois vertices se alinhem no eixo y, falando mais especificamente, se o vertice A for menor que B o algoritmo gira a imagem no sentido horário e vice e versa até os dois alinharem.
# Para a segunda etapa de alinhamento, as linhas detectadas com esse mesmo processo também tem um bom uso, neste caso, após a transformação rigida feito pelo warpAffine agora realizamos uma transformação não rigida na imagem por meio de warpPerspective. A ideia é, dado duas linhas alinhadas em y, detectadas pelo algoritmo hough, julga-se que para um bom alinhamento, as cordenadas das duas pontas devem estar alinhadas em x, com testes foi verificado que um valor possivel, mas não ótimo, de ser usado ser 5 pixels de movimento para cada pixel de diferença entre as cordenadas das duas linhas. Um exemplo de execução: Se a linha superior tiver um pixel a frente da linha inferior, então o algoritmo deslocará 5 pixels para a esquerda os cantos superiores e 5 pixels para a direita os cantos inferiores na imagem final e vice e versa até as duas linhas alinharem.
# O contra destas etapas de transformações é que devido a uma detecção imprecisa das linhas em alguns casos, a estimativa é afetada impossibilitando um resultado melhor alinhado.
# Alguns outros detalhes sobre estas tecnicas encontram-se em minha breve aula sobre Image Stitching presente em: https://github.com/MarcosSoares10/ImageStitchingOpencvandDetailed


#Edge enhance
def sharpen_img(img):
    kernel = np.array([[-1,-1,-1,-1,-1],
                    [-1,2,2,2,-1],
                    [-1,2,8,2,-1],
                    [-2,2,2,2,-1],
                    [-1,-1,-1,-1,-1]])/8.0
    result=cv2.filter2D(img,-1,kernel)
    return result

def euclidian_distance(l):
  result = math.sqrt((l[2] - l[0])**2+(l[3] - l[1])**2)
  return result

def execute_rotation(img,angle):
  h, w = img.shape
  center = w // 2, h // 2
  matrix = cv2.getRotationMatrix2D(center, angle, 1)
  rotated = cv2.warpAffine(img, matrix, (w, h), flags=cv2.INTER_NEAREST, borderValue=255)
  return rotated

def execute_transformation(img,aux,top_steps,bottom_steps):
  #Taking the height and width of source and target image
  height_src_img,width_src_img = img.shape
  height_tgt_img,width_tgt_img = aux.shape

  #Taking the corners of the images: top-left, bottom-left, bottom-right, top-right
  array_corners_source_img = np.float32([[0,0],[0,height_src_img],[width_src_img,height_src_img],[width_src_img,0]])
  array_corners_target_img = np.float32([[top_steps,0],[0+bottom_steps,height_tgt_img],[width_tgt_img+bottom_steps,height_tgt_img],[width_tgt_img+top_steps,0]])
     
  # Apply Perspective Transform Algorithm 
  matrix = cv2.getPerspectiveTransform(array_corners_source_img, array_corners_target_img) 
  result = cv2.warpPerspective(img, matrix, (width_tgt_img,height_tgt_img),flags=cv2.INTER_NEAREST, borderValue=255)
  return result

def evaluate_warpingAffine(img):
  ksize = (10, 10) 

  final_angle = 0
  angle = 0
  execute = True

  aux = cv2.blur(img, ksize)
  aux = sharpen_img(aux)


  edges = cv2.Canny(aux, 50, 200, None, 3)
  hough_lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, None, 50, 10)


  hough_distances = []

  if hough_lines is not None:
      for i in range(0, len(hough_lines)):
          hough_distances.append((euclidian_distance(hough_lines[i][0]),i))


  hough_distances.sort()
  hough_distances.reverse()

  l = hough_lines[hough_distances[0][1]][0]

  while(execute):

    hough_distances = []
    #Verify if aligned on horizontal
    if l[1] > l[3]:
      angle = -1
      final_angle += angle
    elif l[1] < l[3]:
      angle = 1
      final_angle += angle
    else:
      execute = False
    edges = execute_rotation(edges,angle)
    hough_lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, None, 50, 10)
    if hough_lines is not None:
      for i in range(0, len(hough_lines)):
          hough_distances.append((euclidian_distance(hough_lines[i][0]),i))
    hough_distances.sort()
    hough_distances.reverse()
    l = hough_lines[hough_distances[0][1]][0]

  return execute_rotation(img,final_angle),hough_distances,hough_lines,edges

def evaluate_warpingPerspective(img,hough_distances,hough_lines,edges):
   

  final_top_steps = 0
  final_bottom_steps = 0
  
  top_steps = 0
  bottom_steps = 0
  execute = True
  aux = np.zeros((edges.shape[0]+80,edges.shape[1]+80), np.uint8)

  l = hough_lines[hough_distances[0][1]][0]
  l1 = hough_lines[hough_distances[1][1]][0]

  while(execute):
    hough_distances = []
    #Verify if aligned on Vertical and horizontal
    #also, condition execute transformation just if distance in from A to B in x is greater than
    if (l[0] > l1[0]) and ((l[0] - l1[0]) > 3) and (l[1] < l[3]):
      top_steps  += 5
      bottom_steps -= 5
      final_top_steps = top_steps
      final_bottom_steps = bottom_steps
    elif (l[0] < l1[0]) and  ((l1[0] - l[0]) > 3) and (l[1] > l[3]):
      top_steps -= 5
      bottom_steps += 5
      final_top_steps = top_steps
      final_bottom_steps = bottom_steps
    else:
      execute = False
    
    edges = execute_transformation(edges,aux,top_steps,bottom_steps) 
    hough_lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, None, 50, 10)
    
    if hough_lines is not None:
      for i in range(0, len(hough_lines)):
          hough_distances.append((euclidian_distance(hough_lines[i][0]),i))
    
    hough_distances.sort()
    hough_distances.reverse()
    
    l = hough_lines[hough_distances[0][1]][0]
    l1 = hough_lines[hough_distances[1][1]][0]


  return execute_transformation(img,aux,final_top_steps,final_bottom_steps)
  
def execute_ALL(image_directory):

  pathImages = os.listdir(image_directory)
  pathImages.sort()

  for index in range(len(pathImages)):
    img = cv2.imread(image_directory+"/"+pathImages[index])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    aux = binarize_Su_et_al(gray)

    rotated_img,hough_distances,hough_lines,edges = evaluate_warpingAffine(aux)
    rotated_img = evaluate_warpingPerspective(rotated_img,hough_distances,hough_lines,edges)
    


    name, ext = os.path.splitext(pathImages[index])
    cv2.imwrite("denoised_data/"+name+"_denoised_"+ext, rotated_img)




In [84]:
execute_ALL("noisy_data")